# 1. Introduction

In this project, we'll build a multilabel classifier to classify Disaster event related messages into appropriate categories. This can can help tell us about the nature of the event so that the message can be routed to the correct organizations, enabling faster mobilization of resources.

The project will be divided into two separate modules: "Extract, Transformer, Load" and Machine Learning. As you'll see, the initial data for the project is not clean. We'll take this opportunity to showcase some basic ETL skills, and save the clean data to a Sqlite database, which can then be loaded by the ML pipeline. Finally, we will train an XGBoost classifier to predict the labels fo new messages.

This is a walkthrough notebook of the project. The finished scripts, and a web app with a user interface to allow message classification can be found [here](https://github.com/alex-coch/alex-coch.github.io/tree/main/message).

*Note: This project is available as part of some Udacity nanodegrees.*

In [2]:
import pandas as pd
import matplotlib
from sqlalchemy import create_engine

# 2. Data

Data is available to us in two different files: `disaster_messages.csv` and `disaster_categories.csv`.

Let's load these two datasets and take a look at their first few rows.


In [3]:
categories = pd.read_csv('https://raw.githubusercontent.com/alex-coch/alex-coch.github.io/main/message/data/disaster_categories.csv')
messages = pd.read_csv('https://raw.githubusercontent.com/alex-coch/alex-coch.github.io/main/message/data/disaster_messages.csv')

In [ ]:
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


We're only interested in the `message` column from the `disaster_messages` dataset. We'll use that column to train our message classifier, and ignore the other columns.

In [ ]:
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [ ]:
categories.iloc[0]['categories']

'related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'

The `disaster_categories` dataset contains the category labels for our messages, but in a serialized fashion. We have also taken a closer look at the category for the first row. We'll need to convert the categories to a format better suited for our ML model.

# 3. Extract, Transform, Load!

Let's start by merging the two datasets so that the messages and categories are present in the same dataframe.

In [ ]:
df = messages.merge(categories, on='id')

In [ ]:
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### Cleaning and transforming the categories column 

Let's start by splitting the  categories column into separate columns for each category.

In [ ]:
categories = df['categories'].str.split(';',expand=True)

In [ ]:
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


Extracting the column names from a row of the categories dataframe.

In [ ]:
category_column_names = categories.iloc[0].apply(lambda x: x.split("-")[0])

In [ ]:
# Let's check which categories we have
print([name for name in category_column_names])

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


Using the extracted column names as the header of the categories dataframe:

In [ ]:
categories.columns = category_column_names

In [ ]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


Next, we'll fix the values of the above dataset so that they are binary. 1 indicates that a message belonging to a given category.

In [ ]:
# df1 = pd.DataFrame(categories['related'].str[-1])
# df1[(df1.related!='1') & (df1.related!='2')& (df1.related!='0')]

In [ ]:
for column in categories:
    # the last character of a value indicates its binary label
    # print(column)
    categories[column] = categories[column].str[-1].astype(int)

The `related` column contains values other than 1 or 2, so we will convert the 2s to 1s by testricting the maximum value of a category as 1.

In [ ]:
categories['related'].value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

In [ ]:
categories['related'] = categories['related'].clip(0,1)

In [ ]:
categories['related'].value_counts()

1    20246
0     6140
Name: related, dtype: int64

Let's concatenate the categories dataframe back to our original dataframe.

In [ ]:
# drop the original categories column from `df`
df.drop('categories',axis=1,inplace=True)

# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df,categories], axis=1)


In [ ]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### Removing duplicates and NaN rows, if any

In [ ]:
# checking for duplicate rows in df
df.duplicated().sum()

171

In [ ]:
# drop duplicates
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
# drop NaN messages
df.dropna(subset=["message"], axis=0, inplace=True) # drop the row

In [ ]:
# drop the id and original columns as they are not useful for the learning problem
df.drop(["id", "original"], axis=1, inplace=True)

### Saving to a database

As part of this project's simulated scenario, we'll use sqlite and save the dataframe to a db.

In [ ]:
database_filename = "disaster_db"

engine = create_engine('sqlite:///'+ database_filename)
df.to_sql('messages', engine, if_exists="replace", index=False)

# 4. Machine Learning Pipeline

Let's start by loading the data back form our database:

In [ ]:
import re

import nltk
# uncomment below if NLTK data needs to be downloaded
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
engine = create_engine('sqlite:///' + database_filename)
df = pd.read_sql_query('select * from messages', engine)
# the training data is a numpy array of all messages
X = df['message'].values
# the labels are all the different categories
Y = df.drop(columns=['message','genre'], axis=1)
category_names = Y.columns

In [ ]:
len(X)

26215

In [ ]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'],
      dtype=object)

In [ ]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Splitting the dataset into train/test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
print(len(X_train), len(X_test))

20972 5243


### Preprocessing the messages texts

For the classifier to perform well, we need to preprocess the disaster message texts to standardize and tokenize them. We can define a function to do this, which can be used later when defining our model pipeline:

In [ ]:
punctuation_regex = re.compile(r"[^\w\s]")
stopwords = stop_words = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
pos_tags_to_lemmatize = ["n", "v"]

In [ ]:
def tokenize(text: str) -> list:
    """
    Tokenizes a given text.
    Args:
        text: text string
    Returns:
        tokens: list of tokens
    """
    # lowercase string and remove punctuation
    text = punctuation_regex.sub(" ", text.lower()).strip()
    # tokenize text
    tokens = [token for token in word_tokenize(text)]
    # lemmatize text based on pos tags
    for pos_tag in pos_tags_to_lemmatize:
        tokens = [wordnet_lemmatizer.lemmatize(token, pos=pos_tag) for token in word_tokenize(text)]
    # remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    return tokens

### Building the model pipeline

In [ ]:
def build_model() -> GridSearchCV:
    """
    Builds classification model 
    Returns:
        cv: model
    """

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier(learning_rate=0.1)))
    ])

    parameters = {
        "clf__estimator__max_depth": [8, 16],
        "clf__estimator__colsample_bytree":[0.5, 0.75]
    }

    cv = GridSearchCV(pipeline, cv=3, param_grid=parameters, n_jobs=1, scoring="f1_micro")
    return cv

### Fitting the model

In [ ]:
model = build_model()

In [ ]:
model.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fedf9858ca0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBClassifier()))]),
             n_jobs=1,
             param_grid={'clf__estimator__colsample_bytree': [0.5, 0.75],
                         'clf__estimator__max_depth': [8, 16]},
             scoring='f1_micro')

### Evaluating the model

In [ ]:
y_preds = model.predict(X_test)
print(classification_report(y_preds, y_test.values, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.96      0.82      0.89      4685
               request       0.56      0.78      0.65       661
                 offer       0.00      0.00      0.00         0
           aid_related       0.67      0.77      0.72      1901
          medical_help       0.27      0.65      0.38       173
      medical_products       0.31      0.67      0.42       120
     search_and_rescue       0.21      0.55      0.30        49
              security       0.02      0.67      0.05         3
              military       0.37      0.62      0.46        99
           child_alone       0.00      0.00      0.00         0
                 water       0.71      0.75      0.73       321
                  food       0.76      0.80      0.78       594
               shelter       0.59      0.80      0.68       357
              clothing       0.46      0.76      0.57        50
                 money       0.23      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
# collect accuracy scores in a dict
category_name_2_accuracy_score = {}
for i in range(len(category_names)):
    category_name_2_accuracy_score[y_test.columns[i]] = accuracy_score(y_test.values[:,i],y_preds[:,i])
print(pd.Series(category_name_2_accuracy_score))

related                   0.810986
request                   0.894907
offer                     0.994469
aid_related               0.778562
medical_help              0.929430
medical_products          0.957849
search_and_rescue         0.975968
security                  0.984742
military                  0.972535
child_alone               1.000000
water                     0.965669
food                      0.948312
shelter                   0.949266
clothing                  0.989128
money                     0.978447
missing_people            0.989891
refugees                  0.970628
death                     0.968720
other_aid                 0.870875
infrastructure_related    0.932481
transport                 0.962426
buildings                 0.960900
electricity               0.982644
tools                     0.994278
hospitals                 0.988747
shops                     0.994469
aid_centers               0.987984
other_infrastructure      0.951364
weather_related     

This is an imbalanced dataset, as most categories for a given message will be 0. The f1-score is a better metric of the model's performance.

### Predicting categories for new messages

In [ ]:
def predict(text: str) -> list:
    """
    Returns a list of predicted categories for the given text
    Args:
        text: text string
    Returns:
        predicted_categories: list of  categories
    """
    preds = model.predict([text])
    predicted_categories = [category for i, category in enumerate(y_test.columns) if preds[0][i] == 1]
    return predicted_categories

In [ ]:
predict("after the floods in our area we are trapped. we need food and shelter ")

['related',
 'request',
 'aid_related',
 'search_and_rescue',
 'food',
 'shelter',
 'weather_related',
 'floods',
 'direct_report']

# 5. Further Improvements


This notebook sticks to the basics in order to provide a good baseline model for this classification task. There is plenty of room for improvement here, including but not limited to:

1. Using word embeddings (GloVe, word2vec) or even sentence embeddings (Univsersal Sentgence Encoder) to transform message text, instead of using a CountVectorizer. This should allow the model to generalize well to similar/unseen words and improve the model accuracy too.
2. Some of the category column values like `related` and `child_alone` are highly skewed. We can look at adding more data for these categories.
3. A Neural Network can be trained for this task. Better yet, a pre-trained state-of-the-art transformer based network can be fine tuned using the data available to us.
4. Certain classification categories are "noisy" such as "related" or "child alone" (no positive examples). These can either be removed, or more data can be acquired that provides enough missing examples for such cases.

# 6. Summary

In this project, we built an ETL pipeline to load messy and unsuitable-for-training data, clean and transform it, and save it to a database. We also built a ML pipeline to tokenize message text, and train an XGBoost classifier to classify messages into different categories. 

We stuck to the basics for building this classifier, and there's plenty of room for improvement in the future using modern NLP architectures.